In [46]:
%%cpp -d
#pragma cling add_include_path("/usr/include")
#pragma cling add_library_path("/usr/lib")
#pragma cling load("Hist")
#pragma cling load("Tree")
#pragma cling load("RIO")
#pragma cling load("Core")
#pragma cling load("Graf")

#include <vector>
#include <string>
#include <iostream>
#include <cmath>
#include <TFile.h>
#include <TTree.h>
#include <TKey.h>
#include <TDirectoryFile.h>
#include <TH1F.h>
#include <TIterator.h>

using namespace std;

std::vector<std::string> filepaths = {
    "AO2D.root", "AO2D_798.root", "AO2D_797.root", "AO2D_796.root",
    "AO2D_795.root", "AO2D_794.root", "AO2D_793.root", "AO2D_792.root",
    "AO2D_791.root", "AO2D_790.root"
};

double px_calc(double pT, double phi) { return pT * std::cos(phi); }
double py_calc(double pT, double phi) { return pT * std::sin(phi); }
double pz_calc(double pT, double eta) { return pT * std::sinh(eta); }

double calc_abs_momentum(double pT, double phi, double eta) {
    double px = px_calc(pT, phi);
    double py = py_calc(pT, phi);
    double pz = pz_calc(pT, eta);
    return std::sqrt(px * px + py * py + pz * pz);
}

double f1(double dEdx, double pT, double phi, double eta) {
    return 1 / (0.08 * pow(calc_abs_momentum(pT, phi, eta) - 0.4, 1.3)) + 78;
}
double f2(double dEdx, double pT, double phi, double eta) {
    return 1 / (0.17 * pow(calc_abs_momentum(pT, phi, eta) - 0.3, 1)) + 68;
}
double f3(double dEdx, double pT, double phi, double eta) {
    return 1 / (0.5 * pow(calc_abs_momentum(pT, phi, eta) - 0.2, 0.7)) + 63;
}
double f4(double dEdx, double pT, double phi, double eta) {
    return 1 / (5 * pow(calc_abs_momentum(pT, phi, eta) - 0.1, 0.4)) + 60;
}

void process_momentum_variables() {
    // Create output file first
    TFile *outfile = new TFile("ParticlePt.root", "RECREATE");
    
    TH1F *pr_pT = new TH1F("pr_pT", "Protons pT", 300, 0.2, 1);
    TH1F *kaon_pT = new TH1F("kaon_pT", "Kaons pT", 300, 0.0, 1.5);
    TH1F *pi_pT = new TH1F("pi_pT", "Pions pT", 300, 0.1, 0.45);

    for (const auto &filepath : filepaths) {
        TFile *inputfile = TFile::Open(filepath.c_str());
        if (!inputfile || inputfile->IsZombie()) {
            cerr << "Error: cannot open file " << filepath << endl;
            continue;
        }

        TIter next(inputfile->GetListOfKeys());
        TKey *key;

        while ((key = (TKey *)next())) {
            if (strcmp(key->GetClassName(), "TDirectoryFile") == 0) {
                TDirectoryFile *dir = (TDirectoryFile *)key->ReadObj();
                auto treeReducedTrack = (TTree*) dir->Get("O2reducedtrack");
                auto treeBarrelPID = (TTree*) dir->Get("O2rtbarrelpid");
                auto treeRtBarrel = (TTree*) dir->Get("O2rtbarrel");

                if (!treeReducedTrack || !treeBarrelPID || !treeRtBarrel) continue;

                float pT, phiangle, eta, dEdx;
                unsigned char findable;

                treeReducedTrack->SetBranchAddress("fPt", &pT);
                treeReducedTrack->SetBranchAddress("fPhi", &phiangle);
                treeReducedTrack->SetBranchAddress("fEta", &eta);
                treeBarrelPID->SetBranchAddress("fTPCSignal", &dEdx);
                treeRtBarrel->SetBranchAddress("fTPCNClsFindable", &findable);

                Long64_t nEntries = std::min({treeReducedTrack->GetEntries(),
                                              treeBarrelPID->GetEntries(),
                                              treeRtBarrel->GetEntries()});

                for (Long64_t iEntry = 0; iEntry < nEntries; ++iEntry) {
                    treeReducedTrack->GetEntry(iEntry);
                    treeBarrelPID->GetEntry(iEntry);
                    treeRtBarrel->GetEntry(iEntry);

                    if (std::abs(std::tan(phiangle)) < 1e-6 || findable <= 100) continue;

                    int bin = pr_pT->FindBin(pT);
                    double binWidth = pr_pT->GetXaxis()->GetBinWidth(bin);

                    if (f2(dEdx, pT, phiangle, eta) < dEdx && dEdx < f1(dEdx, pT, phiangle, eta)) {
                        pr_pT->Fill(pT, 1.0 / binWidth);
                    } else if (f3(dEdx, pT, phiangle, eta) < dEdx && dEdx < f2(dEdx, pT, phiangle, eta)) {
                        kaon_pT->Fill(pT, 1.0 / binWidth);
                    } else if (f4(dEdx, pT, phiangle, eta) < dEdx && dEdx < f3(dEdx, pT, phiangle, eta)) {
                        pi_pT->Fill(pT, 1.0 / binWidth);
                    }
                }
            }
        }

        inputfile->Close();
        delete inputfile;
    }

    pr_pT->Write();
    kaon_pT->Write();
    pi_pT->Write();
    
    outfile->Write();
    outfile->Close();
    delete outfile;
    cout << "Analysis completed! Results saved to ParticlePt.root" << endl;
}

input_line_174:22:26: error: redefinition of 'filepaths'
std::vector<std::string> filepaths = {
                         ^
input_line_45:22:26: note: previous definition is here
std::vector<std::string> filepaths = {
                         ^
input_line_174:28:8: error: redefinition of 'px_calc'
double px_calc(double pT, double phi) { return pT * std::cos(phi); }
       ^
input_line_45:28:8: note: previous definition is here
double px_calc(double pT, double phi) { return pT * std::cos(phi); }
       ^
input_line_174:29:8: error: redefinition of 'py_calc'
double py_calc(double pT, double phi) { return pT * std::sin(phi); }
       ^
input_line_45:29:8: note: previous definition is here
double py_calc(double pT, double phi) { return pT * std::sin(phi); }
       ^
input_line_174:30:8: error: redefinition of 'pz_calc'
double pz_calc(double pT, double eta) { return pT * std::sinh(eta); }
       ^
input_line_45:30:8: note: previous definition is here
double pz_calc(double pT, double eta) { r

In [47]:
process_momentum_variables();


Analysis complete. Results saved to ParticlePt.root


Error in <TROOT::WriteTObject>: The current directory (PyROOT) is not associated with a file. The object (pr_pT) has not been written.
Error in <TROOT::WriteTObject>: The current directory (PyROOT) is not associated with a file. The object (kaon_pT) has not been written.
Error in <TROOT::WriteTObject>: The current directory (PyROOT) is not associated with a file. The object (pi_pT) has not been written.


In [67]:
void PlotMomentumAnalysis() {
    // Open the input file
    TFile *file = TFile::Open("ParticlePt.root");
    if (!file || file->IsZombie()) {
    std::cerr << "Error: Could not open ParticlePt.root" << std::endl;
        return;
    }
    // load the histograms
    TH1F *pr_pT = (TH1F*)file->Get("pr_pT");
    TH1F *kaon_pT = (TH1F*)file->Get("kaon_pT");
    TH1F *pi_pT = (TH1F*)file->Get("pi_pT");
    if (!pr_pT || !kaon_pT || !pi_pT) {
        std::cerr << "Error: Histograms not found!" << std::endl;
        file->Close();
        return;
    }
    
    gStyle->SetOptStat(0);
    gStyle->SetPalette(kDarkBodyRadiator);

    // // // //   // // // //   // // // //   // // // //   // // // //
    TCanvas *cPrPt = new TCanvas("cPrPt", "Proton pT Fit", 800, 600);
    pr_pT->SetLineColor(kRed);
    pr_pT->GetYaxis()->SetTitle("dN/dp_{T} [(1/(GeV/c))]");
    pr_pT->GetXaxis()->SetTitle("p_{T} [GeV/c]");
    
    pr_pT->Draw();
    pr_pT->GetXaxis()->SetLimits(0, 1.5);
    pr_pT->GetYaxis()->SetLimits(0, 5e6);
    
    TLegend *leg1 = new TLegend(0.6, 0.7, 0.88, 0.88);
    
    leg1->AddEntry(pr_pT, "Proton p_{T}", "l");
    leg1->Draw();
    
    cPrPt->Update();
    cPrPt->SaveAs("proton transversmomentum density.png");

    
    // // // //   // // // //   // // // //   // // // //   // // // //
    TCanvas *cKaonPt = new TCanvas("cKaonPt", "Kaons pT (Main Peak Fit)", 800, 600);
    
    kaon_pT->SetLineColor(kGreen+2);
    kaon_pT->GetYaxis()->SetTitle("dN/dp_{T} [(1/(GeV/c))]");
    kaon_pT->GetXaxis()->SetTitle("p_{T} [GeV/c]");
    kaon_pT->Draw();
    kaon_pT->GetXaxis()->SetLimits(0, 1.5);
    kaon_pT->GetYaxis()->SetLimits(0, 3500000);
    
    TLegend *leg2 = new TLegend(0.6, 0.7, 0.88, 0.88);
    leg2->AddEntry(kaon_pT, "Kaon p_{T}", "l");
    leg2->Draw();
    
    cKaonPt->Update();
    cKaonPt->SaveAs("kaon transversmomentum density.png");

    // // // //   // // // //   // // // //   // // // //   // // // //
    TCanvas *cPiPt = new TCanvas("cPiPt", "Pions pT (Main Peak Fit)", 800, 600);
    pi_pT->SetLineColor(kBlue);
    pi_pT->GetYaxis()->SetTitle("dN/dp_{T} [(1/(GeV/c))]");
    pi_pT->GetXaxis()->SetTitle("p_{T} [GeV/c]");
    pi_pT->Draw();
    pi_pT->GetXaxis()->SetLimits(0, 1.5);
    pi_pT->GetYaxis()->SetLimits(0, 3500000);
    
    TLegend *leg3 = new TLegend(0.6, 0.7, 0.88, 0.88);
    leg3->AddEntry(pi_pT, "Pion p_{T}", "l");
    leg3->Draw();
    
    cPiPt->Update();
    cPiPt->SaveAs("pion transversmomentum density.png");

    // // // //   // // // //   // // // //   // // // //   // // // //
    TCanvas *AllPlots = new TCanvas("AllParticles", "pT distribution for all particles", 800, 600);
    
    pr_pT->SetLineColor(kRed);
    pr_pT->SetMarkerColor(kRed);
    pr_pT->SetMarkerStyle(21);
    pr_pT->SetMarkerSize(0.1);
    
    kaon_pT->SetLineColor(kGreen+2);
    kaon_pT->SetMarkerColor(kGreen+2);
    kaon_pT->SetMarkerStyle(21);
    kaon_pT->SetMarkerSize(0.1);
   
   
    pi_pT->SetLineColor(kBlue);
    pi_pT->SetMarkerColor(kBlue);
    pi_pT->SetMarkerStyle(21);
    pi_pT->SetMarkerSize(0.1);
  
    
    double xmin = 0.0;
    double xmax = 1.5;    
    double ymin = 0.0;
    double ymax = 5e6;

    
    pi_pT->Draw("E1");
    kaon_pT->Draw("E1 SAME");
    pr_pT->Draw("E1 SAME");
    
    pi_pT->GetXaxis()->SetLimits(xmin, xmax);
    pi_pT->GetYaxis()->SetLimits(ymin, ymax);
    
    pi_pT->SetTitle("Transverse momentum distributions; p_{T} [GeV/c]; dN/dp_{T} [GeV/c]^{-1}");
    pi_pT->GetXaxis()->CenterTitle();
    pi_pT->GetYaxis()->CenterTitle();
    
    TLegend *leg = new TLegend(0.65, 0.65, 0.88, 0.88);
    leg->SetHeader("Particles", "C");
    leg->AddEntry(pi_pT, "Pions", "l");
    leg->AddEntry(kaon_pT, "Kaons", "l");
    leg->AddEntry(pr_pT, "Protons", "l");
    leg->SetBorderSize(1);
    leg->SetFillStyle(1);
    leg->Draw();
    
    AllPlots->Update();
    AllPlots->SaveAs("all_particles_pT_distributions.png");

}
 PlotMomentumAnalysis();

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cPrPt
Info in <TCanvas::Print>: png file proton transversmomentum density.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cKaonPt
Info in <TCanvas::Print>: png file kaon transversmomentum density.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cPiPt
Info in <TCanvas::Print>: png file pion transversmomentum density.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: AllParticles
Info in <TCanvas::Print>: png file all_particles_pT_distributions.png has been created
